# Цель проекта проверить:

## 1. Зависит ли вероятность оплаты от выбранного пользователем уровня сложности?

## 2. Существует ли разница во времени между событиями выбора уровня сложности и оплаты для разных групп пользователей с разным уровнем сложности?
### Проверка производится на данных за 2018 год

In [ ]:
#Загрузка необходимой библиотеки
import pandas as pd

In [ ]:
#Загрузка данных
df=pd.read_csv('7_4_Events.csv')
#Выделим данные за 2018 год
cond = (df.start_time>='2018-01-01') & (df.start_time<'2019-01-01') & (df.event_type=='registration')
registered = df[cond]['user_id'] # список пользователей, зарег. в 2018
df_events = df[df.user_id.isin(registered)]
df_events.start_time = pd.to_datetime(df_events.start_time, format='%Y-%m-%dT%H:%M:%S')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [ ]:
#Знакомство с данными
df_events.head()

,id,event_type,selected_level,start_time,tutorial_id,user_id
51405,80308,registration,NaN,2018-01-01 03:48:40,NaN,27832
51406,80309,registration,NaN,2018-01-01 04:07:25,NaN,27833
51407,80310,registration,NaN,2018-01-01 08:35:10,NaN,27834
51408,80311,registration,NaN,2018-01-01 11:54:47,NaN,27835
51409,80312,registration,NaN,2018-01-01 13:28:07,NaN,27836


In [ ]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66959 entries, 51405 to 118364
Data columns (total 6 columns):
id                66959 non-null int64
event_type        66959 non-null object
selected_level    8342 non-null object
start_time        66959 non-null datetime64[ns]
tutorial_id       32954 non-null float64
user_id           66959 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 3.6+ MB


Датафрейм df_events содержит 66959 строк и 6 столбцов, при этом в столбцах selected_level и tutorial_id есть пропущенные значения.
Это обусловлено тем, что не все из параметров selected_level, tutorial_id присутствуют в каждом из событий. Эти параметры заполнятся в зависимости от event_type.

In [ ]:
#Оценим характеристики каждого столбца
df_events.describe(include='all')

,id,event_type,selected_level,start_time,tutorial_id,user_id
count,66959.000000,66959,8342,66959,32954.000000,66959.000000
unique,NaN,5,3,66809,NaN,NaN
top,NaN,registration,medium,2018-03-13 09:37:43,NaN,NaN
freq,NaN,19926,4645,9,NaN,NaN
first,NaN,NaN,NaN,2018-01-01 03:48:40,NaN,NaN
last,NaN,NaN,NaN,2019-01-01 05:50:36,NaN,NaN
mean,113787.000045,NaN,NaN,NaN,40532.934393,37781.543362
std,19329.542752,NaN,NaN,NaN,5213.486632,5751.497904
min,80308.000000,NaN,NaN,NaN,31505.000000,27832.000000
25%,97047.500000,NaN,NaN,NaN,36008.250000,32849.000000


In [ ]:
#Столбцы с типом события и уровнем сложности есть всего 5 и 3 соответственно уникальных значений, посмотрим на них
print('Типы событий: {}, уровни сложностей: {}.'.format(df_events['event_type'].unique(), df_events['selected_level'].unique()))

Типы событий: ['registration' 'tutorial_start' 'tutorial_finish' 'level_choice'
 'pack_choice'], уровни сложностей: [nan 'medium' 'hard' 'easy'].


In [ ]:
#Также оценим, какое количество пользователей совершали события
df_events['user_id'].nunique()

19926

In [ ]:
#Загрузка данных о покупках
df2=pd.read_csv('purchase.csv')
#Выделение данных за 2018 год
df_purchases = df2[df2.user_id.isin(registered)]
df_purchases.event_datetime = pd.to_datetime(df_purchases.event_datetime, format='%Y-%m-%dT%H:%M:%S')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [ ]:
#Знакомство с данными
df_purchases.head()

,id,user_id,event_datetime,amount
1171,16845,27845,2018-01-03 18:53:43,100
1172,16846,27865,2018-01-04 14:46:10,250
1174,16848,27911,2018-01-07 08:19:12,50
1175,16849,27910,2018-01-07 12:11:34,100
1176,16850,27940,2018-01-07 13:16:41,200


In [ ]:
df_purchases.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600 entries, 1171 to 2778
Data columns (total 4 columns):
id                1600 non-null int64
user_id           1600 non-null int64
event_datetime    1600 non-null datetime64[ns]
amount            1600 non-null int64
dtypes: datetime64[ns](1), int64(3)
memory usage: 62.5 KB


В датафрейме с данными о покупках за 2018 год присутствует 4 столбца и 1600 строк, пропущенны значений нет

In [ ]:
#Переименуем стоблцы перед объединением
df_events=df_events.rename(columns={'id':'event_id'})
df_purchases=df_purchases.rename(columns={'id':'purchase_id'})
df_purchases['event_type']='purchase'
#Объединение таблиц
total_df=pd.concat([df_events,df_purchases], sort = False)

In [ ]:
#Сформируем группы пользователей по выбранному уровню сложности
users_with_easy_lvl=total_df[total_df['selected_level'] == 'easy']['user_id'].unique()
users_with_medium_lvl=total_df[total_df['selected_level'] == 'medium']['user_id'].unique()
users_with_hard_lvl=total_df[total_df['selected_level'] == 'hard']['user_id'].unique()

In [ ]:
percent1=len(df_purchases[df_purchases['user_id'].isin(users_with_easy_lvl)])/len(users_with_easy_lvl)
percent2=len(df_purchases[df_purchases['user_id'].isin(users_with_medium_lvl)])/len(users_with_medium_lvl)
percent3=len(df_purchases[df_purchases['user_id'].isin(users_with_hard_lvl)])/len(users_with_hard_lvl)
print('Процент пользователей, совершивших покупку (среди тех, кто выбрал простой уровень сложности): {:.2%}'.format(percent1))
print('Процент пользователей, совершивших покупку (среди тех, кто выбрал средний уровень сложности): {:.2%}'.format(percent2))
print('Процент пользователей, совершивших покупку (среди тех, кто выбрал тяжелый уровень сложности): {:.2%}'.format(percent3))

Процент пользователей, совершивших покупку (среди тех, кто выбрал простой уровень сложности): 7.72%
Процент пользователей, совершивших покупку (среди тех, кто выбрал средний уровень сложности): 20.86%
Процент пользователей, совершивших покупку (среди тех, кто выбрал тяжелый уровень сложности): 35.39%


По полученным данным видно, что чем сложнее уровень выбирает пользователь, тем более вероятно, что он совершит покупку в дальнейшем.

In [ ]:
total_df.head()

,event_id,event_type,selected_level,start_time,tutorial_id,user_id,purchase_id,event_datetime,amount
51405,80308.0,registration,NaN,2018-01-01 03:48:40,NaN,27832,NaN,NaT,NaN
51406,80309.0,registration,NaN,2018-01-01 04:07:25,NaN,27833,NaN,NaT,NaN
51407,80310.0,registration,NaN,2018-01-01 08:35:10,NaN,27834,NaN,NaT,NaN
51408,80311.0,registration,NaN,2018-01-01 11:54:47,NaN,27835,NaN,NaT,NaN
51409,80312.0,registration,NaN,2018-01-01 13:28:07,NaN,27836,NaN,NaT,NaN


In [ ]:
#Сформируем таблицы с датами выбора уровня сложности
easy_lvl_events=total_df[total_df['selected_level'] == 'easy'][['user_id','start_time']].rename(columns={'start_time':'selected_level_time'})
medium_lvl_events=total_df[total_df['selected_level'] == 'medium'][['user_id','start_time']].rename(columns={'start_time':'selected_level_time'})
hard_lvl_events=total_df[total_df['selected_level'] == 'hard'][['user_id','start_time']].rename(columns={'start_time':'selected_level_time'})

In [ ]:
#Проверим данные на повторение пользователей
print(easy_lvl_events.user_id.value_counts().mean(), medium_lvl_events.user_id.value_counts().mean(), hard_lvl_events.user_id.value_counts().mean())

1.0 1.0 1.0


Повторов нет, тк в среднем каждый пользователь встречается ровно 1 раз

In [ ]:
#Сформируем таблицы с датами покупок
easy_lvl_purchase=total_df[(total_df['event_type'] == 'purchase')&(total_df['user_id'].isin(users_with_easy_lvl))][['user_id','event_datetime']].rename(columns={'event_datetime':'purchase_time'})
medium_lvl_purchase=total_df[(total_df['event_type'] == 'purchase')&(total_df['user_id'].isin(users_with_medium_lvl))][['user_id','event_datetime']].rename(columns={'event_datetime':'purchase_time'})
hard_lvl_purchase=total_df[(total_df['event_type'] == 'purchase')&(total_df['user_id'].isin(users_with_hard_lvl))][['user_id','event_datetime']].rename(columns={'event_datetime':'purchase_time'})

In [ ]:
#Объединяем таблицы по номеру пользователя
mergered1=easy_lvl_purchase.merge(easy_lvl_events, on = 'user_id', how = 'inner')
mergered2=medium_lvl_purchase.merge(medium_lvl_events, on = 'user_id', how = 'inner')
mergered3=hard_lvl_purchase.merge(hard_lvl_events, on = 'user_id', how = 'inner')
#Вычисляем разницу между событиями выбора уровня и покупки
mergered1['time_delta'] = mergered1['purchase_time'] - mergered1['selected_level_time']
mergered2['time_delta'] = mergered2['purchase_time'] - mergered2['selected_level_time']
mergered3['time_delta'] = mergered3['purchase_time'] - mergered3['selected_level_time']

In [ ]:
print('Через {} в среднем совершает покупку пользователь, выбравший легкий уровень сложности\nЧерез {} в среднем совершает покупку пользователь, выбравший средний уровень сложности\nЧерез {} в среднем совершает покупку пользователь, выбравший тяжелый уровень сложности'. format(mergered1['time_delta'].mean(), mergered2['time_delta'].mean(), mergered3['time_delta'].mean()))

Через 3 days 14:58:52.941798 в среднем совершает покупку пользователь, выбравший легкий уровень сложности
Через 3 days 23:14:13.165118 в среднем совершает покупку пользователь, выбравший средний уровень сложности
Через 3 days 07:20:41.420814 в среднем совершает покупку пользователь, выбравший тяжелый уровень сложности


Исходя из приведенного выше, можно сделать вывод, что в среднем пользователи совершают покупку через 3 дня.
Нельзя выделить какую-то зависимость между уровнем и быстротой совершения покупки.
Но быстрее всех покупку совершают пользователи, выбравшие тяжелый уровень сложности.

# Вывод:
### Процент пользователей, совершивших покупку (среди тех, кто выбрал простой уровень сложности): 7.72%. Процент пользователей, совершивших покупку (среди тех, кто выбрал средний уровень сложности): 20.86%. Процент пользователей, совершивших покупку (среди тех, кто выбрал тяжелый уровень сложности): 35.39%
### По этим данным можно сделать вывод, что чем сложнее уровень выбирает пользователь, тем вероятнее он совершит покупку платных вопросов. 
### Пользователи, выбравшие легкий уровень, совершают покупку в среднем через 3 дня 14 часов. Пользователи, выбравшие средний уровень, совершают покупку в среднем через 3 дня 23 часа. Пользователи, выбравшие тяжелый уровень, совершают покупку в среднем через 3 дня 7 часов.
### Нельзя выявить прямую зависимость между выбором уровня сложности и быстротой совершения покупки. Но быстрее всех покупку совершают пользователи, выбравшие тяжелый уровень сложности.